In [ ]:
from heapq import heappush, heappop, heapify
import heapq
from collections import defaultdict
import turtle

# Huffman Compression Algorithm
### Explored by: Tihomir Todorov
The goal of this project is to examine and implement the **Huffman algorithm** for compressing and decompressing data.

## Table of Contents
1. [Introduction](#introduction)
    1. [Basics of Encoding](#encoding_basics)
    2. [Types of Compression](#types_of_compression)
    3. [What is Entropy in Data Compression?](#entropy)
2. [Introduction to Huffman Encoding](#huffman_encoding) 
    1. [Construction of Huffman Trees](#huffman_trees)
    2. [Traversing the Huffman Tree](#huffman_traversing)
3. [Huffman Compression in Python](#huffman_python)
    1. [Simple Text Compression](#simple_text)
    2. [Huffman Tree Visualization](#huff_viz)
4. [Pros and Cons of Huffman Compression](#adv_and_disadv)
5. [Real-life Applications of Huffman Encoding](#real_life_app)

<a name="introduction"></a>
## 1. Introduction 

<a name="encoding_basics"></a>
### A. Basics of Encoding

Encoding means to convert the text in some other format.We generally perform encoding for reducing the size of the file.Suppose a text file is available then we can convert the text file in some other format which might be taking less space i.e. we can convert each character in the file in the form of binary string. Converting in the form of binary string would be beneficial because to store a binary 0 or 1 we need only one bit but to store a character we need 8 bits.

**There are two types of encoding:**

1. **Fixed-Length Encoding:** In this method, each character is represented by a fixed length binary codes.
2. **Variable-Length Encoding:** In this method, each character is represented by a variable length binary codes.


Both the encoding techniques would definetly reduce the size of the file but Variable-Length encoding is better than Fixed-Length encoding.Let's understand this with an example.Suppose there is a file for which we have created a table representing the frequency of each character as follows:

| Symbol | A | B | C | D | E | F |
| :------ | :--- | :--- | :--- | :--- | :--- | :--- |
| Count | 45 | 11 | 6 | 16 | 100 | 50 |

**"Fixed-Length Encoding"**

With this approach we assign each character binary codes of fixed length. Since there are 6 characters we need 3 bits to store each character uniquely. So, total bits required to store the whole file is: **(45 + 11 + 6 + 16 + 100 + 50) = 684 bits.**

**"Variable-Length Encoding"**
Since in this method each chracter is being assigned variable length binary code so what we try here is to assign frequent characters short code words and unfrequent characters long code words. Consider the scheme: **a = 10, b = 100, c = 101, d = 110, e = 1, f = 11**

In this case, total bits required to store the whole file is **(45.2 + 11.3 + 6.3 + 16.3 + 100.1 + 50.2) = 389 bits.**

One can easily see the difference between the spaces required to store a file using
Fixed-Length Encoding and Variable-Length Encoding.So,we can conclude that Variable-Length Encoding is much better.

<a name="types_of_compression"></a>
### B. Types of Compression

- There are two main types of compression
 - **lossy**
 - And **lossless**
- The most common type of compression is the **lossless compression**, where the image stays exactly the same after compression and no information is lost.
* On the contrary the **lossy compression** produces images which might be slightly different before compression and after uncompressing.
 * In this process some information might be lost, and the file size can be reduced significantly.
 * Often while taken human anatomy in consideration (We can't tell the difference between certain values of color) we can use **lossy compression** if it isn't very noticeable.

#### Here's an example of lossy and lossless compression. (We can certainly notice the difference in this one, for example look at the lightning):
<a href="https://imgur.com/keenaJd"><img src="https://i.imgur.com/keenaJd.jpg" title="source: imgur.com" /></a>

<a name="entropy"></a>
### C. What is Entropy in Data Compression?

* In information theory, an **entropy coding** (or **entropy encoding**) is a lossless data compression scheme that is independent of the specific characteristics of the medium.
    <br></br>
* One of the main types of entropy coding creates and assigns a unique prefix-free code to each unique symbol that occurs in the input. These entropy encoders then compress data by replacing each fixed-length input symbol with the corresponding variable-length prefix-free output codeword. The length of each codeword is approximately proportional to the negative logarithm of the probability. Therefore, the most common symbols use the shortest codes. 
    <br></br>
* According to [Shannon](https://en.wikipedia.org/wiki/Claude_Shannon), the entropy of an information source S is defined as:
    
    $H(S) = \eta = \sum_{i} p_i log_2 \frac{{\textstyle i}}{{\textstyle p_i}}$
    
    where $p_i$ is the probability that symbol $S_i$ in $S$ will occur.
    <br></br>
    * $log_2 \frac{{\textstyle 1}}{{\textstyle p_i}}$ indicates the amount of information contained in $S_i$, i.e, the number of bits needed to code $S_i$
    <br></br>
    
    * For example, in an image with uniform distribution of gray-level intensity, i.e. $p_i = 1/256$, then the number of bits needed to code each gray level is 8 bits. **The entropy of the image is 8**.
<br></br>

Knowing this as a basis, let's now explore what is **Huffman Encoding**...

<a name="huffman_encoding"></a>
### 2. Introduction to Huffman Encoding

#### The facts
* Developed by [David Huffman](https://en.wikipedia.org/wiki/David_Huffman) in 1951, this technique is the basis for all data compression and encoding schemes

* It is a famous greedy algorithm used for lossless data compressing

* It follows a Greedy approach, since it deals with generating minimum length prefix-free binary codes

* It uses variable-length encoding scheme for assigning binary codes to characters depending on how frequently they occur in the given text. The character that occurs most frequently is assigned the smallest code and the one that occurs least frequently gets the largest code



<a name="huffman_trees"></a>
### A. Construction of Huffman Trees

In this section we will explore the steps for creating the **Huffman Tree** for the following characters along with their frequencies:

| Symbol | a | e | i | o | u | s | t |
| :------ | :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| Frequency | 10 | 15 | 12 | 3 | 4 | 13 | 1 |

* **Step 1**: Create leaf nodes for all the characters and add them to the **min heap** (see [Binary Heaps](https://en.wikipedia.org/wiki/Binary_heap)). 
    * **Step A**: 
Create a leaf node for each character and build a min heap using all the nodes (The frequency value is used to compare two nodes in min heap)

        <a href="https://static.studytonight.com/data-structures/images/leaf-node.PNG"><img src="https://static.studytonight.com/data-structures/images/leaf-node.PNG" title="source: imgur.com" /></a>
<br></br>

* **Step 2**: Repeat the following steps till heap has more than one nodes
    * **Step B**: Extract two nodes, say **x** and **y**, with minimum frequency from the heap
    * **Step C**: Create a new internal node **z** with **x** as its left child and **y** as its right child. Also: 
    
    $$frequency(z) = frequency(x) + frequency(y)$$
    <br></br>
    * **Step D**: Add z to min heap
        * Extract and Combine nodes **o** and **t** with an internal node having 4 as the frequency
        * Add the new internal node to priority queue:
        
        <a href="https://static.studytonight.com/data-structures/images/combining-node.PNG"><img src="https://static.studytonight.com/data-structures/images/combining-node.PNG" title="source: imgur.com" /></a>

        * Extract and Combine node **u** with an internal node having 8 as the frequency
        * Add the new internal node to priority queue:
        
        <a href="https://static.studytonight.com/data-structures/images/combining-node1.PNG"><img src="https://static.studytonight.com/data-structures/images/combining-node1.PNG" title="source: imgur.com" /></a>    
        * Extract and Combine node **a**
        * Add the new internal node to priority queue:
        <a href="https://static.studytonight.com/data-structures/images/combining-node2.PNG"><img src="https://static.studytonight.com/data-structures/images/combining-node2.PNG" title="source: imgur.com" /></a>    
        * Extract and Combine nodes **i** and **s**
        * Add the new internal node to priority queue:
        <a href="https://static.studytonight.com/data-structures/images/combining-node3.PNG"><img src="https://static.studytonight.com/data-structures/images/combining-node3.PNG" title="source: imgur.com" /></a>   
        * Extract and Combine nodes **e** with an internal node having 18 as the frequency
        * Add the new internal node to priority queue:
        <a href="https://static.studytonight.com/data-structures/images/combining-node4.PNG"><img src="https://static.studytonight.com/data-structures/images/combining-node4.PNG" title="source: imgur.com" /></a>  
        * Finally, Extract and Combine internal nodes having 25 and 33 as the frequency
        * Add the new internal node to priority queue
        <a href="https://static.studytonight.com/data-structures/images/combining-node5.PNG"><img src="https://static.studytonight.com/data-structures/images/combining-node5.PNG" title="source: imgur.com" /></a>  
        
* **Step 3**: Since internal node with frequency 58 is the only node in the queue, it becomes the root of our **Huffman Tree**


<a name="huffman_traversing"></a>
### B. Traversing the Huffman Tree
In this section we will explore the steps for traversing the Huffman Tree.

* **Step 1**: Create an auxiliary array
* **Step 2**: Traverse the tree starting from root node
* **Step 3**: Add 0 to array while traversing the left child and add 1 to array while traversing the right child
* **Step 4**: Show the array elements whenever a leaf node is found

Following the above steps for the Huffman tree that we generated in the previous section, we get prefix-free and variable-length binary codes with minimum expected codeword length:
<a href="https://static.studytonight.com/data-structures/images/binary-huffman.PNG"><img src="https://static.studytonight.com/data-structures/images/binary-huffman.PNG" title="source: imgur.com" /></a>    

| Symbol  | i | s | e | u | t | o | a |
| :------ | :------ | :------ | :------ | :------ | :------ | :------ | :------ |
| Binary Codes | 00 | 01 | 10 | 1100 | 11010 | 11011 | 111 |

Using the above binary codes:
* Suppose the string "aeioust" needs to be transmitted from computer A (sender) to computer B (receiver) across a network. Using concepts of Huffman encoding, the string gets encoded to **"11110001101111000111010"(111 | 10 | 00 | 11011 | 1100 | 01 | 11010)** at the sender side.
    * Once received at the receiver's side, it will be decoded back by traversing the Huffman tree. For decoding each character, we start traversing the tree from root node. Start with the first bit in the string. A ‘1’ or ‘0’ in the bit stream will determine whether to go left or right in the tree. Print the character, if we reach a leaf node.
<br></br>  
* Thus for the above bit stream:
<a href="https://static.studytonight.com/data-structures/images/decoding.PNG"><img src="https://static.studytonight.com/data-structures/images/decoding.PNG" title="source: imgur.com" /></a> 

Following this path:
* 111 gets decoded to **'a'**
* 10 gets decoded to **'e'**
* 00 gets decoded to **'i'**
* 11011 gets decoded to **'o'**
* 1100 gets decoded to **'u'**
* 01 gets decoded to **'s'**
* 11010 gets decoded to **'t'**

<a name="huffman_python"></a>
## 3. Huffman Compression in Python

<a name="simple_text"></a>
### A. Simple Text Compression
Now let's implement what we've learned in Python code...


In [ ]:

text = input("Enter text: ")

#Let's begin by creating the function to perform the compression
def encode(frequency):
    heap = [[weight, [symbol, ""]] for symbol, weight in frequency.items()]
    heapify(heap)
    while len(heap) > 1:
        lo = heappop(heap)
        hi = heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    return sorted(heappop(heap)[1:], key=lambda p: (len(p[-1]), p))
 
frequency = defaultdict(int)
for ch in text:
    frequency[ch] += 1

#Then we will encode the text and assign it to the variable huff
huff = encode(frequency)

#This part prints an overview of the characters, and assigns the binary values to a string
print("Symbol\tWeight\tHuffman Code")
[print("%s\t%s\t%s" % (p[0], frequency[p[0]], p[1])) for p in huff]

bitstring = ""
for ch in text:
    for item in huff:
        if ch in item:
            bitstring += item[1]

#Here we will show the overview of our compression and the binary string of the text
binary = ((bin(int(bitstring, base = 2))))
print("Orginal size:", len(text) * 7, "bits")
print("Compressed size:", len(binary) - 2, "bits")
print("This is saving ", (len(text) * 7) - (len(binary) - 2)," bits, with approximately",round((len(text) * 7) / (len(binary) - 2)), ": 1 compression ratio.")
print('Binary code: ',binary[2:])

#Finally let's perform the decompression and show the output of it
decode = binary[2:]
decodestr = ""
code = ""
for digit in bitstring:
    code += digit
    pos = 0
    for p in huff:
        if code == p[1]:
            decodestr += huff[pos][0]
            code = ""
        pos += 1
        
print("Decompressed text: " + decodestr)

<a name="huff_viz"></a>
### B. Huffman Tree Visualization

Upgrading the code from the previous section, let's try now to draw the actual Huffman Tree, using the my favourite turtle. 

In [ ]:
'''Let's build up on the previous code, but this time with a using OOP'''

class Huffman:
    def __init__(self, data) -> None:
        self.data = data
        self.origin = 0, 250

        if len(set(data)) <= 2:
            raise ValueError('not enough unique characters')

    def encode(self,frequency):
        heap = [[weight, [symbol,'']] for symbol, weight in frequency.items()]
        heapq.heapify(heap)
        while len(heap) > 1:
            lo = heapq.heappop(heap)
            hi = heapq.heappop(heap)
            for pair in lo[1:]:
                pair[1] = '0' + pair[1]
            for pair in hi[1:]:
                pair[1] = '1' + pair[1]
            heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
        return sorted(heapq.heappop(heap)[1:], key=lambda p: (len(p[-1]), p))
    
    def extract_info(self):
        frequency = defaultdict(int)
        for symbol in self.data:
            frequency[symbol] += 1

        huff = self.encode(frequency)
        return frequency, huff
        

    #This is where the drawing part goes
    def turtle_space(self):
        turtle.penup()
        turtle.forward(20)
        turtle.pendown()

    def down_lt_branch(self, a, length):
        branch_length=length*(length*6/2**a)
        turtle.forward(10)
        turtle.right(90)
        turtle.forward(branch_length)
        turtle.left(90)
        turtle.forward(10)
        self.turtle_space()

    def down_rt_branch(self, a, length):
        branch_length=length*(length*6/2**a)
        turtle.forward(10)
        turtle.left(90)
        turtle.forward(branch_length)
        turtle.right(90)
        turtle.forward(10)
        self.turtle_space()
    
    def write_uncircled(self,character,size):
        turtle.penup()
        turtle.left(90)
        turtle.backward(size*2)
        turtle.pendown()
        turtle.write(character,font=("Helvetica",15,"normal"))
        turtle.penup()
        turtle.forward(size*2)
        turtle.right(90)
        turtle.pendown()

    def write_circled(self,character,size):
        turtle.penup()
        turtle.left(90)
        turtle.backward(size*3)
        turtle.pendown()
        turtle.write(character,font=("Helvetica",15,"normal"))
        turtle.penup()
        turtle.forward(size*3)
        turtle.pendown()
        turtle.circle(12)
        turtle.right(90)

    def init_turtle_screen(self):
        turtle.title('huffman tree')
        turtle.speed(0)
        turtle.color("#56B6C4")
        turtle.bgcolor("#06114F")
        turtle.setup(width=1.0, height=1.0)
    
    def tp(self):
        turtle.penup()
        turtle.goto(self.origin)
        turtle.pendown()


    #data processing
    def huffman_tree_info(self):
        frequency, huff = self.extract_info()

        huff_frequency = [[frequency[symbol], binary] for symbol, binary in huff] #making addition simpler
        parent_nodes = [] #for drawing the parent nodes

        huff_frequency.sort(key=lambda x:len(x[1]), reverse=True)
        
        current_binary_value = huff_frequency[0]

        while len(huff_frequency) > 2:
            for freq, binary in huff_frequency:
                if binary[:-1] == current_binary_value[1][:-1] and binary != current_binary_value[1]:
                    parent_value = freq + current_binary_value[0]
                    huff_frequency.append([parent_value, binary[:-1]])

                    parent_nodes.append([parent_value, binary[:-1]])

                    huff_frequency.remove(current_binary_value)
                    huff_frequency.remove([freq, binary])
            current_binary_value = huff_frequency[0]
            huff_frequency.sort(key=lambda x:len(x[1]), reverse=True)
        
        huffman_tree_data = huff + parent_nodes
        '''
        if first element of list is str: it's a character
        if first element of list is int: it's a parent node
        '''
        return huffman_tree_data

    def draw_tree(self):
        self.init_turtle_screen()

        _, huff = self.extract_info()

        length=len(huff[len(huff)-1][1])

        huffman_tree_data = self.huffman_tree_info()

        #starting off with the main branch
        self.tp()
        turtle.right(90)
        self.write_circled(len(self.data), 2)

        for char, binary_code in huffman_tree_data:
            for index, bit in enumerate(binary_code):
                if bit == '0':
                    self.down_lt_branch(index, length)
                else:
                    self.down_rt_branch(index, length)
            
            if type(char) is str:
                if char == ' ':
                    self.write_uncircled("space", 5)
                self.write_uncircled(char, len(char))
            else:
                self.write_circled(char, len(str(char)))
            self.tp()
        try:
            turtle.penup()
            turtle.goto(0,300)
            turtle.pendown()
            turtle.write(self.data, font=("Helvetica",25,"normal"), align='center')

            turtle.hideturtle()
            turtle.mainloop()
        except:
            turtle.Terminator


'''
Huffman(Input text with more than 2 unique characters) It works well, with very short sentences, 
otherwise the graph starts looking messy as you'll probably see.

If you run the program more than once, you will probably encounter Terminator error 
(that I unsuccessfully tried to avoid). Just run it once more and it will work :)
'''

if __name__ == '__main__':
    huffman = Huffman(input("Enter the desired text: "))
    huffman.draw_tree()

<a name="adv_and_disadv"></a>
## 4. Pros and Cons of Huffman Compression

### A. Pros
* This encoding scheme results in saving lot of storage space, since the binary codes generated are variable in length

* It generates shorter binary codes for encoding symbols/characters that appear more frequently in the input string

* The binary codes generated are prefix-free

### B. Cons

* Lossless data encoding schemes, like Huffman encoding, achieve a lower compression ratio compared to lossy encoding techniques. Thus, lossless techniques like Huffman encoding are suitable only for encoding text and program files and are unsuitable for encoding digital images.

* Huffman encoding is a relatively slower process since it uses two passes - one for building the statistical model, and another for encoding. Thus, the lossless techniques that use Huffman encoding are considerably slower than others.

* Since length of all the binary codes is different, it becomes difficult for the decoding software to detect whether the encoded data is corrupt. This can result in an incorrect decoding and subsequently, a wrong output.

<a name="real_life_app"></a>
## 5. Real-life Applications of Huffman Encoding

* Huffman encoding is widely used in compression formats like **GZIP**, **PKZIP** (winzip) and **BZIP2**.

* Multimedia codecs like **JPEG**, **PNG** and **MP3** uses Huffman encoding (to be more precised the prefix codes)

* Huffman encoding still dominates the compression industry since newer arithmetic and range coding schemes are avoided due to their patent issues.